<a href="https://colab.research.google.com/github/ksetdekov/trip_choice_optimizer/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

from  scipy.stats import gamma, norm
from mvsampling.mvsampling import HandsTable

In [2]:
# np.random.seed(7336) # фиксируем seed

# risk tolerance
$$\rho = 3.37$$

# initialize

In [3]:
mu_hat_i = 0 # mean
t_i = 0
alpha_i = 0.5
beta_i = 0.5
# rho = 3.37
rho = 1

In [4]:
def update_mean(X, T_last, mu_last):
    mu_new = T_last * mu_last / (T_last + 1) + X / (T_last + 1)
    return mu_new

def update_samples(T):
    return T + 1

def update_shape(a):
    return a + 0.5

def update_rate(X, mu_last, beta_last, T_last):
    beta_new = beta_last + (T_last / (T_last + 1)) * (np.square(X - mu_last)) / 2
    return beta_new

In [5]:
hands = pd.DataFrame({'name': ['трамвай + 944', 'с799 + пешком', 'пешком + 944', 'мотоцикл', 'с799 + 944', 
                               'рыжая + жёлтая + салатовая', 'трамвай + салатовая',
                               'каршеринг', 'серая + салатовая', 'рыжая + фиолетовая + салатовая', 'рыжая + салатовая',
                               'машина', 'самокат', '2гис', 'яндекс карты', 'гугл карты', 'citymapper', 'транспорт москвы',
                               'свой велосипед', 'московский велосипед']})

In [6]:
hands['mu'] = 0
hands['Te'] = 0
hands['alpha'] = 0.5
hands['beta'] = 0.5

In [7]:
def to_minutes(timestr: str):
  '''
  convert timestr to float minutes
  '''
  return pd.to_timedelta(timestr).total_seconds()/60

In [8]:
def update_hands(hands, name, value):
    if isinstance(value, str):
        try:
            value = to_minutes(value)
        except ValueError:
            raise ValueError('input time string in hh:mm:ss format')
    elif isinstance(value, float) or isinstance(value, int):
        pass
    else:
        raise ValueError('input time string or int/float value')
    # print(value)
    _, mu, t, alpha, beta = hands[hands.name ==name].values[0]
    beta = update_rate(value, mu, beta, t)
    mu =  update_mean(value, t, mu)
    t = update_samples(t)
    alpha = update_shape(alpha)

    # print(mu, t, alpha, beta)

    hands.loc[hands.name ==name, 'mu'] = mu
    hands.loc[hands.name ==name, 'Te'] = t
    hands.loc[hands.name ==name, 'alpha'] = alpha
    hands.loc[hands.name ==name, 'beta'] = beta

    return hands


In [9]:
hands = update_hands(hands, 'мотоцикл', 20)
hands = update_hands(hands, 'рыжая + жёлтая + салатовая', 47.63)
hands = update_hands(hands, 'каршеринг', 31.75)
hands = update_hands(hands, 'рыжая + салатовая', 52.6)
hands = update_hands(hands, 'каршеринг', 23.15)
hands = update_hands(hands, 'рыжая + фиолетовая + салатовая', 52.82)
hands = update_hands(hands, 'серая + салатовая', 52.37)
hands = update_hands(hands, 'мотоцикл', 21.05)
hands = update_hands(hands, 'мотоцикл', 26.40)
hands = update_hands(hands, 'мотоцикл', "0:25:18")
hands = update_hands(hands, 'мотоцикл', "0:24:54")
hands = update_hands(hands, 'мотоцикл', "0:26:51")
hands = update_hands(hands, 'каршеринг', "0:28:04")
hands = update_hands(hands, 'каршеринг', "0:28:05")
hands = update_hands(hands, 'мотоцикл', "0:24:54")
hands = update_hands(hands, 'мотоцикл', "0:26:58")
hands = update_hands(hands, 'мотоцикл', "0:26:29")
hands = update_hands(hands, 'машина', "0:23:59")
hands = update_hands(hands, 'каршеринг', "0:36:41")
hands = update_hands(hands, 'машина', "0:24:49")
hands = update_hands(hands, 'машина', "0:21:56")
hands = update_hands(hands, 'машина', "0:22:03")
hands = update_hands(hands, 'машина', "0:23:21")
hands = update_hands(hands, 'мотоцикл', "0:27:45")
hands = update_hands(hands, 'машина', "0:22:26")
hands = update_hands(hands, 'машина', "0:25:00")
hands = update_hands(hands, 'машина', "0:24:07")
hands = update_hands(hands, 'машина', "0:23:38")
hands = update_hands(hands, 'машина', "0:24:13")
hands = update_hands(hands, 'машина', "0:25:32")
hands = update_hands(hands, 'машина', "0:22:11")
hands = update_hands(hands, 'самокат', "0:47:17")
hands = update_hands(hands, 'машина', "0:23:33")
hands = update_hands(hands, 'машина', "0:22:54")
hands = update_hands(hands, 'машина', "0:22:42")
hands = update_hands(hands, 'каршеринг', "0:33:02")
hands = update_hands(hands, 'машина', "0:23:31")
hands = update_hands(hands, 'машина', "0:24:39")
hands = update_hands(hands, 'машина', "0:24:16")
hands = update_hands(hands, 'машина', "0:28:12")
hands = update_hands(hands, 'трамвай + 944', "0:41:35")
hands = update_hands(hands, 'машина', "0:26:27")
hands = update_hands(hands, 'машина', "0:23:55")
hands = update_hands(hands, 'с799 + 944', "0:45:48")
hands = update_hands(hands, 'машина', "0:32:23")
hands = update_hands(hands, 'машина', "0:22:34")
hands = update_hands(hands, 'машина', "0:25:00")
hands = update_hands(hands, 'машина', "0:27:01")
hands = update_hands(hands, 'каршеринг', "0:31:14")
hands = update_hands(hands, 'трамвай + салатовая', "0:48:40")
hands = update_hands(hands, 'пешком + 944', "0:46:56")
hands = update_hands(hands, 'машина', "0:31:17")
hands = update_hands(hands, 'каршеринг', "0:34:08")
hands = update_hands(hands, 'машина', "0:31:18")
hands = update_hands(hands, 'машина', "0:27:10")
hands = update_hands(hands, 'каршеринг', "0:25:51")
hands = update_hands(hands, 'транспорт москвы', "0:49:36")
hands = update_hands(hands, 'яндекс карты', "0:44:21")
hands = update_hands(hands, 'с799 + пешком', "0:55:24")
hands = update_hands(hands, 'гугл карты', "0:40:50")
hands = update_hands(hands, '2гис', "0:59:04")
hands = update_hands(hands, 'каршеринг', "0:29:00")
hands = update_hands(hands, 'citymapper', "0:52:31")
hands = update_hands(hands, 'машина', "0:27:36")
hands = update_hands(hands, 'машина', "0:26:23")
hands = update_hands(hands, 'машина', "0:24:04")
hands = update_hands(hands, 'гугл карты', "0:44:06")
hands = update_hands(hands, 'машина', "0:33:59")
hands = update_hands(hands, 'каршеринг', "0:26:42")
hands = update_hands(hands, 'машина', "0:26:36")
hands = update_hands(hands, 'машина', "0:23:13")
hands = update_hands(hands, 'трамвай + 944', "0:48:14")
hands = update_hands(hands, 'машина', "0:25:05")
hands = update_hands(hands, 'машина', "0:30:01")
hands = update_hands(hands, 'каршеринг', "0:30:13")
hands = update_hands(hands, 'машина', "0:23:31")
hands = update_hands(hands, 'с799 + 944', "0:44:02")
hands = update_hands(hands, 'машина', "0:32:43")
hands = update_hands(hands, 'каршеринг', "0:25:07")
hands = update_hands(hands, 'гугл карты', "0:55:16")
hands = update_hands(hands, 'каршеринг', "0:35:15")
hands = update_hands(hands, 'с799 + 944', "0:56:38")
hands = update_hands(hands, 'машина', "0:23:55")
hands = update_hands(hands, 'машина', "0:22:29")
hands = update_hands(hands, 'яндекс карты', "0:47:40")
hands = update_hands(hands, 'каршеринг', "0:25:10")
hands = update_hands(hands, 'машина', "0:27:06")
hands = update_hands(hands, 'каршеринг', "0:30:21")
hands = update_hands(hands, 'машина', "0:23:50")
hands = update_hands(hands, 'машина', "0:21:28")
hands = update_hands(hands, 'машина', "0:26:38")
hands = update_hands(hands, 'машина', "0:21:33")
hands = update_hands(hands, 'машина', "0:25:22")
hands = update_hands(hands, 'каршеринг', "0:29:08")
hands = update_hands(hands, 'каршеринг', "0:22:20")
hands = update_hands(hands, 'машина', "0:22:37")
hands = update_hands(hands, 'машина', "0:23:22")
hands = update_hands(hands, 'машина', "0:23:09")
hands = update_hands(hands, 'каршеринг', "0:26:06")
hands = update_hands(hands, 'машина', "0:26:10")
hands = update_hands(hands, 'машина', "0:29:32")
hands = update_hands(hands, 'каршеринг', "0:27:13")
hands = update_hands(hands, 'каршеринг', "0:26:02")
hands = update_hands(hands, 'машина', "0:31:01")
hands = update_hands(hands, 'машина', "0:23:00")
hands = update_hands(hands, 'каршеринг', "0:38:52")
hands = update_hands(hands, 'машина', "0:23:29")
hands = update_hands(hands, 'машина', "0:22:10")
hands = update_hands(hands, 'машина', "0:24:05")
hands = update_hands(hands, 'машина', "0:23:24")
hands = update_hands(hands, 'каршеринг', "0:31:12")
hands = update_hands(hands, 'рыжая + жёлтая + салатовая', "0:49:05")
hands = update_hands(hands, 'машина', "0:26:40")
hands = update_hands(hands, 'машина', "0:29:12")
hands = update_hands(hands, 'машина', "0:25:00")
hands = update_hands(hands, 'пешком + 944', "0:52:59")
hands = update_hands(hands, 'машина', "0:22:21")
hands = update_hands(hands, 'машина', "0:23:28")
hands = update_hands(hands, 'машина', "0:23:59")
hands = update_hands(hands, 'машина', "0:24:58")
hands = update_hands(hands, 'мотоцикл', "0:26:48")
hands = update_hands(hands, 'машина', "0:23:03")
hands = update_hands(hands, 'каршеринг', "0:31:21")
hands = update_hands(hands, 'машина', "0:21:21")
hands = update_hands(hands, 'машина', "0:24:40")
hands = update_hands(hands, 'каршеринг', "0:29:10")
hands = update_hands(hands, 'машина', "0:23:14")
hands = update_hands(hands, 'трамвай + салатовая', "0:56:49")
hands = update_hands(hands, 'машина', "0:18:30")
hands = update_hands(hands, 'машина', "0:21:26")
hands = update_hands(hands, 'каршеринг', "0:28:32")
hands = update_hands(hands, 'машина', "0:25:20")
hands = update_hands(hands, 'машина', "0:22:53")
hands = update_hands(hands, 'машина', "0:27:19")
hands = update_hands(hands, 'машина', "0:21:56")
hands = update_hands(hands, 'машина', "0:22:18")
hands = update_hands(hands, 'каршеринг', "0:34:07")
hands = update_hands(hands, 'свой велосипед', "0:53:00")
hands = update_hands(hands, 'машина', "0:23:39")
hands = update_hands(hands, 'каршеринг', "0:27:14")
hands = update_hands(hands, 'каршеринг', "0:25:57")
hands = update_hands(hands, 'московский велосипед', "0:52:11")
hands = update_hands(hands, 'каршеринг', "0:40:00")
hands = update_hands(hands, 'машина', "0:21:29")
hands = update_hands(hands, 'машина', "0:27:05")
hands = update_hands(hands, 'машина', "0:25:28")


hands

,name,mu,Te,alpha,beta
0,трамвай + 944,44.908333,2,1.5,11.555625
1,с799 + пешком,55.400000,1,1.0,0.500000
2,пешком + 944,49.958333,2,1.5,9.650625
3,мотоцикл,25.218182,11,6.0,31.720960
4,с799 + 944,48.822222,3,2.0,47.040370
5,рыжая + жёлтая + салатовая,48.356667,2,1.5,1.028044
6,трамвай + салатовая,52.741667,2,1.5,17.105625
7,каршеринг,29.700556,30,15.5,278.091245
8,серая + салатовая,52.370000,1,1.0,0.500000
9,рыжая + фиолетовая + салатовая,52.820000,1,1.0,0.500000


5: for each t = K + 1, K + 2, . . ., do

7: Sample τi,t from Gamma(αi,t−1, βi,t−1).

8: Sample θi,t from N (ˆµi,t−1, 1/Ti,t−1)

9: Play arm i(t) = arg maxi∈[K] ρθi,t − 1/τi,t and observe reward Xi(t),t

10: Update(ˆµi(t),t−1, Ti(t),t−1, αi(t),t−1, βi(t),t−1)

11: end for


In [10]:
tau = gamma.rvs( a = hands.alpha, scale=1/hands.beta)
theta = norm.rvs(hands.mu, 1/hands.Te)
hands['tau'] = tau
hands['theta'] = theta
hands['SD'] = np.sqrt(1/tau)
hands['var95'] = theta + norm.ppf(1-0.05/2) * hands.SD

if hands.mu.min() == 0:
    output1 = hands.reindex(np.argsort(hands.Te))
else:
    # hands['sorter'] = 1/tau + rho * theta
    output1 = hands.reindex(np.argsort(rho * theta + 1/tau))

output1

,name,mu,Te,alpha,beta,tau,theta,SD,var95
11,машина,24.847468,79,40.0,339.105858,0.108087,24.865088,3.041677,30.826665
3,мотоцикл,25.218182,11,6.0,31.720960,0.105205,25.121077,3.083061,31.163765
7,каршеринг,29.700556,30,15.5,278.091245,0.084252,29.712108,3.445158,36.464493
14,яндекс карты,46.008333,2,1.5,3.250069,0.475629,46.229706,1.449992,49.071638
12,самокат,47.283333,1,1.0,0.500000,0.830349,47.692061,1.097412,49.842949
18,свой велосипед,53.000000,1,1.0,0.500000,3.993483,53.042334,0.500408,54.023115
10,рыжая + салатовая,52.600000,1,1.0,0.500000,1.833707,53.003299,0.738474,54.450681
16,citymapper,52.516667,1,1.0,0.500000,3.554160,53.405655,0.530434,54.445287
0,трамвай + 944,44.908333,2,1.5,11.555625,0.108654,44.829939,3.033735,50.775950
19,московский велосипед,52.183333,1,1.0,0.500000,0.611768,52.406506,1.278517,54.912354


## для хилокомода

In [11]:
hands_drops = pd.DataFrame({'name': ['хилокомод', 'гилан', 'гилан ультра комфорт', 'стиллавит',
                                     'артелак баланс', 'артелак всплеск уно', 'артелак всплеск',
                                     'систейн ультра плюс', 'систейн ультра', 'визмед гель',
                                     'визмед лайт', 'хилабак', 'окутиарз'
                                     ]})
hands_drops['mu'] = 0
hands_drops['Te'] = 0
hands_drops['alpha'] = 0.5
hands_drops['beta'] = 0.5
# # с 5 по 20 декабря 2019
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(2/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(24/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(14/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(10/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(16/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# # пропуск  с июля по октябрь 2020
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(30/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(11/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(16/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(19/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(18/10))
# # март 2021
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(17/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(18/10))
# # апрель 2021
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(14/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(20/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# hands_drops = update_hands(hands_drops, 'хилокомод', 30/(12/10))
# # # вышел на работу с сентября
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(11/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(21/12))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(15/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(17/10))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(13/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(32/12))
hands_drops = update_hands(hands_drops, 'гилан', 30/(16/12))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(9/10))
hands_drops = update_hands(hands_drops, 'гилан', 30/(10/12))
hands_drops = update_hands(hands_drops, 'гилан ультра комфорт', 30/(11/12))
hands_drops = update_hands(hands_drops, 'гилан', 30/(12/12))
hands_drops = update_hands(hands_drops, 'хилокомод', 30/(11/10)) # начал Артелак всплеск
hands_drops = update_hands(hands_drops, 'артелак всплеск', 30/(12/10)) # начал визмед гель
hands_drops = update_hands(hands_drops, 'визмед гель', 30/(13/9)) # начал Артелак всплеск уно
hands_drops = update_hands(hands_drops, 'артелак всплеск уно', 30/(15/15)) # начал стиллавит
hands_drops = update_hands(hands_drops, 'стиллавит', 30/(8/10)) # начал артелак баланс 
hands_drops = update_hands(hands_drops, 'артелак баланс', 30/(7/10)) # начал систейн ультра 23 февраля
hands_drops = update_hands(hands_drops, 'систейн ультра', 30/(21/15)) 
hands_drops = update_hands(hands_drops, 'систейн ультра плюс', 30/(18/10)) # 3 апреля начал окутиарз
hands_drops = update_hands(hands_drops, 'окутиарз', 30/(18/10)) # 15 апреля начал визмед лайт
hands_drops = update_hands(hands_drops, 'визмед лайт', 30/(15/10)) # 1 мая  начал хилабак 10 мл
hands_drops = update_hands(hands_drops, 'хилабак', 30/(12/10)) # 12 мая  начал хилабак 10 мл
hands_drops = update_hands(hands_drops, 'хилабак', 30/(12/10)) # 24 мая  начал систейн ультра плюс 10 мл
# hands_drops = update_hands(hands_drops, 'систейн ультра плюс', 30/(18/10)) 
hands_drops

,name,mu,Te,alpha,beta
0,хилокомод,24.525670,7,4.0,82.831148
1,гилан,23.378571,5,3.0,195.451020
2,гилан ультра комфорт,32.727273,1,1.0,0.500000
3,стиллавит,37.500000,1,1.0,0.500000
4,артелак баланс,42.857143,1,1.0,0.500000
5,артелак всплеск уно,30.000000,1,1.0,0.500000
6,артелак всплеск,25.000000,1,1.0,0.500000
7,систейн ультра плюс,16.666667,1,1.0,0.500000
8,систейн ультра,21.428571,1,1.0,0.500000
9,визмед гель,20.769231,1,1.0,0.500000


In [12]:
tau_drops = gamma.rvs( a = hands_drops.alpha, scale=1/hands_drops.beta)
theta_drops = norm.rvs(hands_drops.mu, 1/hands_drops.Te)
hands_drops['tau'] = tau_drops
hands_drops['theta'] = theta_drops
hands_drops['SD'] = np.sqrt(1/tau_drops)
hands_drops['var95'] = theta_drops + norm.ppf(1-0.05/2) * hands_drops.SD


if hands_drops.mu.min() == 0:
    
    output2 = hands_drops.reindex(np.argsort(hands_drops.Te))
else:
    # hands_drops['sorter'] = 1/tau_drops + rho * theta_drops
    output2 = hands_drops.reindex(np.argsort(rho * theta_drops + 1/tau_drops))

output2


,name,mu,Te,alpha,beta,tau,theta,SD,var95
12,окутиарз,16.666667,1,1.0,0.500000,3.379160,16.395803,0.543996,17.462015
7,систейн ультра плюс,16.666667,1,1.0,0.500000,1.287676,17.552116,0.881245,19.279324
10,визмед лайт,20.000000,1,1.0,0.500000,3.633726,20.640385,0.524595,21.668572
8,систейн ультра,21.428571,1,1.0,0.500000,1.281737,20.237028,0.883284,21.968234
9,визмед гель,20.769231,1,1.0,0.500000,1.157995,20.796429,0.929280,22.617784
11,хилабак,25.000000,2,1.5,0.500000,3.019255,26.556417,0.575506,27.684388
6,артелак всплеск,25.000000,1,1.0,0.500000,0.541828,25.105795,1.358530,27.768466
5,артелак всплеск уно,30.000000,1,1.0,0.500000,1.230157,29.337650,0.901612,31.104778
2,гилан ультра комфорт,32.727273,1,1.0,0.500000,1.126165,31.145808,0.942321,32.992724
3,стиллавит,37.500000,1,1.0,0.500000,5.150813,37.667391,0.440618,38.530986


## для поездки с работы на ОТ, минимизуруем минуты

In [13]:
hands_back = HandsTable(['2гис', 'яндекс карты', 'гугл карты', 'citymapper',
                                     'транспорт москвы'
                                     ])
hands_back.update_hands('2гис', '00:55:54')
hands_back.update_hands('яндекс карты', '00:49:41')
hands_back.update_hands('гугл карты', '01:05:54') # салатовая и 38 трамвай
hands_back.update_hands('citymapper', '00:56:12')  # салатовая и 38 трамвай
hands_back.update_hands('транспорт москвы', '00:51:29') # через третьяковскую до шаболовки
hands_back.update_hands('2гис', '01:06:55') # автобус до технопарка, от павелецкой - трамвай
hands_back.update_hands('яндекс карты', '01:04:50') # 799 и 944 автобус
hands_back.update_hands('яндекс карты', '01:02:45') # метро и трамвай

In [14]:
hands_back.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
4,транспорт москвы,51.483333,1,1.0,0.500000,3.133530,50.176715,0.564915,51.283928
3,citymapper,56.200000,1,1.0,0.500000,1.405484,55.975357,0.843504,57.628594
2,гугл карты,65.900000,1,1.0,0.500000,1.855977,63.810799,0.734030,65.249471
1,яндекс карты,59.088889,3,2.0,67.933426,0.014191,58.779746,8.394432,75.232530
0,2гис,61.408333,2,1.5,30.841736,0.009130,61.990669,10.465432,82.502538


## забота о себе

### уровень стресса за день после вечернего

In [15]:
stress_level = HandsTable(['ничего', 'gratitide journal бумажный','gratitide journal бумажный + woebot',
                    'woebot','gratitide journal бумажный + ReWi', 'ReWi'])
stress_level.update_hands('gratitide journal бумажный', 43)
stress_level.update_hands('ничего', 49)
stress_level.update_hands('gratitide journal бумажный + woebot', 34)  # на 26 января
stress_level.update_hands('ничего', 31)  # на 27 января
stress_level.update_hands('gratitide journal бумажный + ReWi', 37)  # на 28 января
stress_level.update_hands('ReWi', 48)  # на 29 января
stress_level.update_hands('woebot', 35)  # на 30 января
stress_level.update_hands('gratitide journal бумажный + woebot', 34)  # на 31 января
stress_level.update_hands('woebot', 33)  # на 1 февраля
stress_level.update_hands('woebot', 32)  # на 2 февраля
stress_level.update_hands('woebot', 38)  # на 3 февраля 
stress_level.update_hands('gratitide journal бумажный + woebot', 28)  # на 4 февраля 
stress_level.update_hands('ничего', 33) # ничего 5
stress_level.update_hands('gratitide journal бумажный + woebot', 33)  # gratitide journal бумажный + woebot 6
stress_level.update_hands('ничего', 38) # ничего 7 и вечером gratitide journal бумажный + woebot
stress_level.update_hands('gratitide journal бумажный + woebot', 27)  # 8
stress_level.update_hands('gratitide journal бумажный + ReWi', 30)  # 9
stress_level.update_hands('ничего', 36) # ничего 10
stress_level.update_hands('woebot', 35) #  11
stress_level.update_hands('ничего', 39) #  12
stress_level.update_hands('ничего', 32) #  13
stress_level.update_hands('ничего', 31) #  14 
stress_level.update_hands('gratitide journal бумажный + woebot', 31)  # 15 
stress_level.update_hands('gratitide journal бумажный + woebot', 33)  # 16
stress_level.update_hands('ничего', 32) #  17
stress_level.update_hands('gratitide journal бумажный + woebot', 32)  # 18
stress_level.update_hands('woebot', 33) #  19
stress_level.update_hands('gratitide journal бумажный + woebot', 32)  # 20
stress_level.update_hands('gratitide journal бумажный + woebot', 39)  # 21
stress_level.update_hands('woebot', 38) #  22
stress_level.update_hands('ничего', 47) #  23
stress_level.update_hands('woebot', 34) #  24
stress_level.update_hands('woebot', 27) #  25
stress_level.update_hands('gratitide journal бумажный + ReWi', 23) #  26
stress_level.update_hands('ничего', 27)  # 27
stress_level.update_hands('woebot', 40)  # 28
stress_level.update_hands('woebot', 29)  # 1 march
stress_level.update_hands('woebot', 24)  # 2 march
stress_level.update_hands('ничего', 30)  # 3
stress_level.update_hands('gratitide journal бумажный', 24) # 4
stress_level.update_hands('gratitide journal бумажный + woebot', 24) # 5
stress_level.update_hands('gratitide journal бумажный + woebot', 28) # 6
stress_level.update_hands('gratitide journal бумажный + woebot', 28) # 7
stress_level.update_hands('gratitide journal бумажный + woebot', 34) # 8
stress_level.update_hands('ReWi', 39) # 9
stress_level.update_hands('ничего', 26) # 10
stress_level.update_hands('ничего', 37) # 11
stress_level.update_hands('ничего', 26) # 12
stress_level.update_hands('gratitide journal бумажный + ReWi', 23)  # 13
stress_level.update_hands('gratitide journal бумажный + woebot', 26) # 14
stress_level.update_hands('gratitide journal бумажный + woebot', 18) # 15
stress_level.update_hands('gratitide journal бумажный', 27) # 16
stress_level.update_hands('woebot', 21)  # 17 march
stress_level.update_hands('gratitide journal бумажный + ReWi', 25)  # 18
stress_level.update_hands('gratitide journal бумажный + ReWi', 24)  # 19
stress_level.update_hands('gratitide journal бумажный + ReWi', 26)  # 20
stress_level.update_hands('gratitide journal бумажный + ReWi', 28)  # 21
stress_level.update_hands('ReWi', 27) # 22
stress_level.update_hands('gratitide journal бумажный', 23) # 23
stress_level.update_hands('gratitide journal бумажный + woebot', 25)  # 24
stress_level.update_hands('gratitide journal бумажный + ReWi', 24)  # 25
stress_level.update_hands('woebot', 25)  # 26 march
stress_level.update_hands('gratitide journal бумажный + woebot', 34)  # 27
stress_level.update_hands('gratitide journal бумажный + ReWi', 37)  # 28
stress_level.update_hands('gratitide journal бумажный + woebot', 24)  # 29
stress_level.update_hands('gratitide journal бумажный + woebot', 18)  # 30.03
stress_level.update_hands('gratitide journal бумажный', 27)  # 31 03
stress_level.update_hands('ничего', 16) # 01.04
stress_level.update_hands('ничего', 33) # 02
stress_level.update_hands('gratitide journal бумажный + ReWi', 26)  # 03
stress_level.update_hands('gratitide journal бумажный', 31)  # 04
stress_level.update_hands('ничего', 25)  # 05.04
stress_level.update_hands('gratitide journal бумажный + ReWi', 29)  # 06
stress_level.update_hands('gratitide journal бумажный', 23)  # 07
stress_level.update_hands('gratitide journal бумажный + ReWi', 27)  # 08
stress_level.update_hands('woebot', 24)  # 09
stress_level.update_hands('gratitide journal бумажный + ReWi', 28)  # 10
stress_level.update_hands('gratitide journal бумажный + ReWi', 25)  # 11
stress_level.update_hands('ничего', 28)  # 12
stress_level.update_hands('gratitide journal бумажный + ReWi', 24)  # 13
stress_level.update_hands('gratitide journal бумажный + ReWi', 24)  # 14
stress_level.update_hands('gratitide journal бумажный + ReWi', 20)  # 15
stress_level.update_hands('ничего', 24)  # 16
stress_level.update_hands('ничего', 23)  # 17
stress_level.update_hands('ничего', 21)  # 18
stress_level.update_hands('gratitide journal бумажный + ReWi', 23)  # 19
stress_level.update_hands('gratitide journal бумажный + ReWi', 24)  # 20
stress_level.update_hands('gratitide journal бумажный + woebot', 28)  # 21
stress_level.update_hands('gratitide journal бумажный + ReWi', 30)  # 22
stress_level.update_hands('gratitide journal бумажный + ReWi', 21)  # 23
stress_level.update_hands('gratitide journal бумажный + ReWi', 34)  # 24
stress_level.update_hands('gratitide journal бумажный + ReWi', 33)  # 25
stress_level.update_hands('woebot', 22)  # 26
stress_level.update_hands('gratitide journal бумажный', 26)  # 27
stress_level.update_hands('gratitide journal бумажный + ReWi', 20)  # 28
stress_level.update_hands('gratitide journal бумажный + ReWi', 27)  # 29
stress_level.update_hands('gratitide journal бумажный + ReWi', 26)  # 30
stress_level.update_hands('gratitide journal бумажный + ReWi', 37)  # 1 мая
stress_level.update_hands('gratitide journal бумажный + ReWi', 34)  # 2 мая
stress_level.update_hands('gratitide journal бумажный + ReWi', 35)  # 3 мая
stress_level.update_hands('gratitide journal бумажный + ReWi', 27)  # 4 мая
stress_level.update_hands('gratitide journal бумажный', 26)  # 5
stress_level.update_hands('gratitide journal бумажный', 24)  # 6 мая
stress_level.update_hands('ReWi', 37)  # 7 мая
stress_level.update_hands('gratitide journal бумажный + ReWi', 26)  # 8 мая
stress_level.update_hands('gratitide journal бумажный + ReWi', 27)  # 9 мая
stress_level.update_hands('gratitide journal бумажный', 27)  # 10
stress_level.update_hands('gratitide journal бумажный', 28)  # 11
stress_level.update_hands('gratitide journal бумажный + woebot', 26)  # 12
stress_level.update_hands('ReWi', 22)  # 13 мая
stress_level.update_hands('ничего', 42)  # 14 мая
stress_level.update_hands('gratitide journal бумажный + ReWi', 40)  # 15
stress_level.update_hands('gratitide journal бумажный + ReWi', 30)  # 16
stress_level.update_hands('gratitide journal бумажный', 26)  # 17
stress_level.update_hands('ReWi', 23)  # 18
stress_level.update_hands('gratitide journal бумажный + ReWi', 24)  # 19
stress_level.update_hands('gratitide journal бумажный', 32)  # 20
stress_level.update_hands('ReWi', 43)  # 21
stress_level.update_hands('ReWi', 30)  # 22
stress_level.update_hands('woebot', 21)  # 23
stress_level.update_hands('gratitide journal бумажный + ReWi', 22)  # 24
stress_level.update_hands('gratitide journal бумажный', 24)  # 25
stress_level.update_hands('gratitide journal бумажный', 25)  # 26
stress_level.update_hands('gratitide journal бумажный + ReWi', 25)  # 27
stress_level.update_hands('ничего', 30)  # 28
stress_level.update_hands('ничего', 16)  # 29
stress_level.update_hands('gratitide journal бумажный', 31)  # 30
stress_level.update_hands('gratitide journal бумажный', 25)  # 31
stress_level.update_hands('gratitide journal бумажный', 29)  # 01.06
stress_level.update_hands('gratitide journal бумажный + ReWi', 32)  # 2
# stress_level.update_hands('gratitide journal бумажный', 29)  # 03.06
# stress_level.update_hands('gratitide journal бумажный', 29)  # 04.06


In [16]:
stress_level.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
1,gratitide journal бумажный,27.421053,19,10.0,192.815789,0.062206,27.450053,4.009457,35.308444
4,gratitide journal бумажный + ReWi,27.615385,39,20.0,494.115385,0.042137,27.616109,4.871550,37.164172
2,gratitide journal бумажный + woebot,28.909091,22,11.5,294.409091,0.034884,28.913193,5.354108,39.407052
3,woebot,30.058824,17,9.0,324.970588,0.032150,30.009278,5.577106,40.940205
0,ничего,30.880000,25,13.0,830.820000,0.023696,30.905240,6.496217,43.637590
5,ReWi,33.625000,8,4.5,320.437500,0.017260,33.630110,7.611758,48.548881


## Бассейн

сколько человек, включая меня/ на число дорожек

In [17]:
pool = HandsTable(["понедельник", "вторник", "среда", "четверг", "пятница"])

понедельник 1830 и пятница - 1 дорожка

пятница - нет второго сеанса

In [18]:
pool.update_hands('вторник', 2/2)
pool.update_hands('четверг', 4/2)
pool.update_hands('понедельник', 3/1)
pool.update_hands('среда', 6/2)
pool.update_hands('понедельник', 2/2)
pool.update_hands('пятница', 3/1)
pool.update_hands('четверг', 6/2)
pool.update_hands('вторник', 9/2)
pool.update_hands('четверг', 7/2)
pool.update_hands('среда', 6/2)
pool.update_hands('пятница', 3/1)
pool.update_hands('понедельник', 3/2)
pool.update_hands('среда', 3/1)
pool.update_hands('понедельник', 5/2)
# pool.update_hands('понедельник', 5/2) 6 июня 


In [19]:

pool.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
0,понедельник,2.000000,4,2.5,1.750000,1.611443,1.904032,0.787757,3.448008
2,среда,3.000000,3,2.0,0.500000,11.127135,2.854471,0.299784,3.442037
3,четверг,2.833333,3,2.0,1.083333,1.443151,2.961273,0.832423,4.592792
4,пятница,3.000000,2,1.5,0.500000,0.466233,3.182554,1.464531,6.052983
1,вторник,2.750000,2,1.5,3.562500,0.308698,3.082363,1.799836,6.609977


## Кондиционер

на сколько градусов поставить утром и когда его выключат?

In [20]:
conditioner = HandsTable([18, 19, 20, 21, 22, 23, 24, 25, 26, 'off'], minimize=False)

In [21]:
conditioner.update_hands(23, (pd.to_timedelta('18:00:00') - pd.to_timedelta('09:00:00')).total_seconds()/3600)
conditioner.update_hands(18, (pd.to_timedelta('10:30:00') - pd.to_timedelta('08:30:00')).total_seconds()/3600)
conditioner.update_hands(26, (pd.to_timedelta('15:07:00') - pd.to_timedelta('09:00:00')).total_seconds()/3600) # жарко
conditioner.update_hands(19, (pd.to_timedelta('09:05:00') - pd.to_timedelta('08:40:00')).total_seconds()/3600) # холодно
conditioner.update_hands(24, (pd.to_timedelta('18:00:00') - pd.to_timedelta('11:30:00')).total_seconds()/3600)
conditioner.update_hands(21, (pd.to_timedelta('12:59:00') - pd.to_timedelta('11:20:00')).total_seconds()/3600)
conditioner.update_hands(25, (pd.to_timedelta('17:00:00') - pd.to_timedelta('09:38:00')).total_seconds()/3600)
conditioner.update_hands('off', (pd.to_timedelta('15:23:00') - pd.to_timedelta('09:00:00')).total_seconds()/3600) # жарко
conditioner.update_hands('off', (pd.to_timedelta('11:00:00') - pd.to_timedelta('09:00:00')).total_seconds()/3600) # hot
conditioner.update_hands(20, (pd.to_timedelta('18:00:00') - pd.to_timedelta('09:00:00')).total_seconds()/3600) 
conditioner.update_hands(20, (pd.to_timedelta('11:55:00') - pd.to_timedelta('09:00:00')).total_seconds()/3600) 
conditioner.update_hands(22, (pd.to_timedelta('16:28:00') - pd.to_timedelta('11:00:00')).total_seconds()/3600) # hot
conditioner.update_hands(23, (pd.to_timedelta('12:30:00') - pd.to_timedelta('08:38:00')).total_seconds()/3600) # hot
conditioner.update_hands(18, (pd.to_timedelta('16:20:00') - pd.to_timedelta('08:30:00')).total_seconds()/3600) # cold
conditioner.update_hands(25, (pd.to_timedelta('10:47:00') - pd.to_timedelta('08:30:00')).total_seconds()/3600) # hot
# conditioner.update_hands(24, (pd.to_timedelta('10:47:00') - pd.to_timedelta('09:08:00')).total_seconds()/3600) # 3 june




In [22]:
conditioner.grade()

,name,mu,Te,alpha,beta,tau,theta,SD,var95
6,24,6.500000,1,1.0,0.500000,0.888407,6.221614,1.060948,4.142194
8,26,6.116667,1,1.0,0.500000,0.393918,6.914059,1.593298,3.791252
4,22,5.466667,1,1.0,0.500000,1.601570,4.811852,0.790182,3.263124
2,20,5.958333,2,1.5,9.751736,0.309900,6.996530,1.796341,3.475766
5,23,6.433333,2,1.5,7.087778,0.315089,6.054325,1.781490,2.562669
7,25,4.825000,2,1.5,6.960069,0.283640,5.243961,1.877659,1.563818
3,21,1.650000,1,1.0,0.500000,0.407415,3.139068,1.566684,0.068423
1,19,0.416667,1,1.0,0.500000,0.994109,1.075610,1.002959,-0.890153
9,off,4.191667,2,1.5,5.303403,0.219786,3.576960,2.133044,-0.603730
0,18,4.916667,2,1.5,9.006944,0.145296,5.350472,2.623449,0.208606
